<a href="https://colab.research.google.com/github/soumithreddyb/BinaryNet/blob/master/CNN_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import keras
from keras.datasets import mnist

#download mnist data and split into train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()

Using TensorFlow backend.


In [0]:
#reshape data to fit model
X_train = X_train.reshape(60000,28,28,1)
X_test = X_test.reshape(10000,28,28,1)

In [0]:
from keras.utils import to_categorical
#one-hot encode target column
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
y_train[0]

array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.], dtype=float32)

In [0]:
# -*- coding: utf-8 -*-
from __future__ import absolute_import
import keras.backend as K


def round_through(x):
    '''Element-wise rounding to the closest integer with full gradient propagation.
    A trick from [Sergey Ioffe](http://stackoverflow.com/a/36480182)
    '''
    rounded = K.round(x)
    return x + K.stop_gradient(rounded - x)


def _hard_sigmoid(x):
    '''Hard sigmoid different from the more conventional form (see definition of K.hard_sigmoid).
    # Reference:
    - [BinaryNet: Training Deep Neural Networks with Weights and Activations Constrained to +1 or -1, Courbariaux et al. 2016](http://arxiv.org/abs/1602.02830}
    '''
    x = (0.5 * x) + 0.5
    return K.clip(x, 0, 1)


def binary_sigmoid(x):
    '''Binary hard sigmoid for training binarized neural network.
    # Reference:
    - [BinaryNet: Training Deep Neural Networks with Weights and Activations Constrained to +1 or -1, Courbariaux et al. 2016](http://arxiv.org/abs/1602.02830}
    '''
    return round_through(_hard_sigmoid(x))


def binary_tanh(x):
    '''Binary hard sigmoid for training binarized neural network.
     The neurons' activations binarization function
     It behaves like the sign function during forward propagation
     And like:
        hard_tanh(x) = 2 * _hard_sigmoid(x) - 1 
        clear gradient when |x| > 1 during back propagation
    # Reference:
    - [BinaryNet: Training Deep Neural Networks with Weights and Activations Constrained to +1 or -1, Courbariaux et al. 2016](http://arxiv.org/abs/1602.02830}
    '''
    return 2 * round_through(_hard_sigmoid(x)) - 1


def binarize(W, H=1):
    '''The weights' binarization function, 
    # Reference:
    - [BinaryNet: Training Deep Neural Networks with Weights and Activations Constrained to +1 or -1, Courbariaux et al. 2016](http://arxiv.org/abs/1602.02830}
    '''
    # [-H, H] -> -H or H
    Wb = H * binary_tanh(W / H)
    return Wb


def _mean_abs(x, axis=None, keepdims=False):
    return K.stop_gradient(K.mean(K.abs(x), axis=axis, keepdims=keepdims))

    
def xnorize(W, H=1., axis=None, keepdims=False):
    Wb = binarize(W, H)
    Wa = _mean_abs(W, axis, keepdims)
    return Wa, Wb

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, BatchNormalization, Activation, MaxPooling2D
#create model
model = Sequential()
#add model layers
model.add(Conv2D(128, kernel_size=3, input_shape=(28,28,1), use_bias=False))
model.add(BatchNormalization(axis=-1, momentum=0.9, epsilon=1e-6, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None))
model.add(Activation(binary_tanh))

model.add(Conv2D(128, kernel_size=3, input_shape=(28,28,1), use_bias=False))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization(axis=-1, momentum=0.9, epsilon=1e-6, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None))
model.add(Activation(binary_tanh))

model.add(Conv2D(256, kernel_size=3, use_bias=False))
model.add(BatchNormalization(axis=-1, momentum=0.9, epsilon=1e-6, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None))
model.add(Activation(binary_tanh))
          
model.add(Conv2D(256, kernel_size=3, use_bias=False))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization(axis=-1, momentum=0.9, epsilon=1e-6, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None))
model.add(Activation(binary_tanh))
        
          
model.add(Flatten())
model.add(Dense(1024, use_bias=False))
model.add(BatchNormalization(axis=-1, momentum=0.9, epsilon=1e-6, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None))
model.add(Activation(binary_tanh))
          
model.add(Dense(10, activation='softmax', use_bias=False))

W0711 09:55:14.600757 139954577635200 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0711 09:55:14.634311 139954577635200 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0711 09:55:14.638785 139954577635200 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0711 09:55:14.689269 139954577635200 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0711 09:55:14.690694 1399545776

In [0]:
#compile model using accuracy to measure model performance
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

W0711 09:55:16.598416 139954577635200 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 128)       1152      
_________________________________________________________________
batch_normalization_1 (Batch (None, 26, 26, 128)       512       
_________________________________________________________________
activation_1 (Activation)    (None, 26, 26, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 128)       147456    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 128)       0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 12, 12, 128)       512       
_________________________________________________________________
activation_2 (Activation)    (None, 12, 12, 128)       0         
__________

In [0]:
#train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3)

W0711 09:55:20.457802 139954577635200 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 60000 samples, validate on 10000 samples
Epoch 1/3
60000/60000 [==============================] - 89s 1ms/step - loss: 0.1663 - acc: 0.9576 - val_loss: 0.0676 - val_acc: 0.9798
Epoch 2/3
60000/60000 [==============================] - 85s 1ms/step - loss: 0.0780 - acc: 0.9784 - val_loss: 0.0575 - val_acc: 0.9860
Epoch 3/3
60000/60000 [==============================] - 83s 1ms/step - loss: 0.0663 - acc: 0.9822 - val_loss: 0.0511 - val_acc: 0.9862


In [0]:
import numpy as np
print(np.shape(model.get_weights()))

(26,)


In [0]:
for we in model.get_weights() :
    print (we.shape)

(3, 3, 1, 128)
(128,)
(128,)
(128,)
(128,)
(3, 3, 128, 128)
(128,)
(128,)
(128,)
(128,)
(3, 3, 128, 256)
(256,)
(256,)
(256,)
(256,)
(3, 3, 256, 256)
(256,)
(256,)
(256,)
(256,)
(4096, 1024)
(1024,)
(1024,)
(1024,)
(1024,)
(1024, 10)


In [0]:
model.layers[0].get_weights()


[array([[[[-0.06450471,  0.03548804,  0.01540588, ..., -0.00105776,
           -0.03934376, -0.0877908 ]],
 
         [[-0.06060199,  0.08164276, -0.01112318, ...,  0.02551278,
           -0.05201096, -0.01735308]],
 
         [[ 0.07112369, -0.03420259,  0.03801239, ...,  0.05262733,
            0.04018071,  0.04103475]]],
 
 
        [[[-0.05186548,  0.08346141, -0.03195366, ..., -0.06872345,
           -0.01437917, -0.07140156]],
 
         [[ 0.01061094,  0.04159461, -0.01406878, ..., -0.08887785,
           -0.02116715,  0.03151202]],
 
         [[-0.01732489, -0.0655929 ,  0.01846346, ..., -0.05706717,
           -0.05447811, -0.06279247]]],
 
 
        [[[ 0.00597985, -0.07020144, -0.05394525, ...,  0.00102679,
            0.05403333,  0.00829383]],
 
         [[ 0.05886688,  0.0020363 ,  0.05009479, ...,  0.08081517,
            0.06722291,  0.06962539]],
 
         [[-0.00706768, -0.02151883, -0.09496803, ..., -0.05707788,
           -0.02813955,  0.04988058]]]], dtype=float32

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# 2. Save Keras Model or weights on google drive



     |████████████████████████████████| 993kB 4.9MB/s 


In [0]:
import sys
sys.path.insert(1, '/Users/<username>/google-cloud-sdk/platform/google_appengine')
sys.path.insert(1, '/Users/<username>/google-cloud-sdk/platform/google_appengine/lib/yaml/lib')
sys.path.insert(1, 'lib')

if 'google' in sys.modules:
    del sys.modules['google']

In [0]:
model.save('model.h5')    
model_file = drive.CreateFile({'title' : 'model.h5'})
model_file.SetContentFile('model.h5')
model_file.Upload()

# download to google drive
drive.CreateFile({'id': model_file.get('id')})

GoogleDriveFile({'id': '1GP6dqxSsBPMRgmPPtNaIO1j5TO9MmAAx'})

In [0]:
model.save_weights('model_weights.h5')
weights_file = drive.CreateFile({'title' : 'model_weights.h5'})
weights_file.SetContentFile('model_weights.h5')
weights_file.Upload()
drive.CreateFile({'id': weights_file.get('id')})

GoogleDriveFile({'id': '1FQuJ9VqKp_i4U8lNnM0UDHP0LMl3yQSJ'})

In [0]:
# 3. reload weights from google drive into the model

# use (get shareable link) to get file id
last_weight_file = drive.CreateFile({'id': '1sj...'}) 
last_weight_file.GetContentFile('last_weights.mat')
model.load_weights('last_weights.mat')

In [0]:
!ls drive

MNIST_CNN_weights.h5
